d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# 3.7 Tables and Views

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this notebook you:<br>
* Differentiate between...
  - Views and tables
  - Global and temporary operations
  - Managed and unmanaged tables
* Write to managed and unmanaged tables
* Explore the effect of dropping tables on the metadata and underlying data

In [0]:
%run ../Includes/Classroom-Setup

Data mounted to /mnt/davis ...

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Tables vs Views<br>

There are a lot of concepts when it comes to different options for handling data in various SQL environments.  One primary disctinction is between a _table_ and _view_:<br><br>

- A **table** creates a table in an existing database
- A **view** is a SQL query stored in a database

When we create a table, we're writing data to a database.  In our case, we're writing to the Databricks File System and storing the metadata in the Hive Metastore, a common metastore in various environments.  A view, by contrast, is just saving the query itself.  We have to recalculate this query each time we call the view, which can take more time but it will also give us the most current data.

Also, views and tables are scoped in different ways:<br><br>

- A **global** view or table is available across all clusters
- A **temporary** view or table is available only in the notebook you're working in

Import the dataset

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsParquet
USING Parquet 
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-1p.parquet"
  )

Write to a temporary view. A **temporary view** is session-scoped and will be dropped when the session ends(when the cluster's terminates).

In [0]:
%sql
DROP VIEW IF EXISTS groupedView;

CREATE OR REPLACE TEMPORARY VIEW groupedView
  AS (
    SELECT count(*) AS count
    FROM fireCallsParquet 
    GROUP BY Call_Type_Group
    ORDER BY count
  )

In [0]:
%sql
SELECT * FROM groupedView

count
82614
466826
475281
970341
2804560


Perform the same command but write to a table instead.

In [0]:
%sql
DROP TABLE IF EXISTS groupedTable;

CREATE TABLE IF NOT EXISTS groupedTable
  AS (
    SELECT count(*) AS count
    FROM fireCallsParquet 
    GROUP BY Call_Type_Group
    ORDER BY count
  )

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM groupedTable

count
82614
466826
475281
970341
2804560


Notice the speed difference between the `CREATE` statements and the reads in the view vs the table.

**Reading data from a temporary view is expensive compared to creating a view. The opposite is true in case of normal tables**

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Managed and Unmanaged Tables<br>

-sandbox
A **managed table** is a table that manages both the data itself as well as the metadata.  In this case, a `DROP TABLE` command removes both the metadata for the table as well as the data itself.  

**Unmanaged tables** manage the metadata from a table such as the schema and data location, but the data itself sits in a different location, often backed by a blob store like the Azure Blob or object store like S3. Dropping an unmanaged table drops only the metadata associated with the table while the data itself remains in place.

<div><img src="https://files.training.databricks.com/images/eLearning/ETL-Part-2/managed-and-unmanaged-tables.png" style="height: 400px; margin: 20px"/></div>

Start with a managed table.

In [0]:
%sql
USE default;

DROP TABLE IF EXISTS tableManaged;

CREATE TABLE tableManaged (
  var1 INT,
  var2 INT
);

INSERT INTO tableManaged
  VALUES (1, 1), (2, 2)

num_affected_rows,num_inserted_rows
2,2


Use `DESCRIBE EXTENDED` to describe the contents of the table.  Scroll down to see the table `Type`.

Notice the location is also `dbfs:/user/hive/warehouse/< your database >/tablemanaged`.

In [0]:
%sql
DESCRIBE EXTENDED tableManaged

col_name,data_type,comment
var1,int,
var2,int,
,,
# Partitioning,,
Not partitioned,,
,,
# Detailed Table Information,,
Name,default.tablemanaged,
Location,dbfs:/user/hive/warehouse/tablemanaged,
Provider,delta,


Now use an external, or unmanaged, table

In [0]:
%python
# Make sure there are no extra files floating around
dbutils.fs.rm("/tmp/unmanagedTable", True)

Out[2]: False

In [0]:
%sql
DROP TABLE IF EXISTS tableUnmanaged;

CREATE EXTERNAL TABLE tableUnmanaged (
  var1 INT,
  var2 INT
)
STORED AS parquet
LOCATION '/tmp/unmanagedTable'

Describe the table and look for the `Type`

In [0]:
%sql
DESCRIBE EXTENDED tableUnmanaged

col_name,data_type,comment
var1,int,null
var2,int,null
,,
# Detailed Table Information,,
Database,default,
Table,tableunmanaged,
Owner,root,
Created Time,Fri Feb 11 22:42:20 UTC 2022,
Last Access,UNKNOWN,
Created By,Spark 3.1.2,


This is an external, or managed table.  If we were to shut down our cluster, this data will persist.  Now insert values into the table.

In [0]:
%sql
INSERT INTO tableUnmanaged
  VALUES (1, 1), (2, 2)

Take a look at the result.

In [0]:
%sql
SELECT * FROM tableUnmanaged

var1,var2
1,1
2,2


Now view the underlying files in where the data was persisted.

In [0]:
%fs ls /tmp/unmanagedTable

path,name,size
dbfs:/tmp/unmanagedTable/_SUCCESS,_SUCCESS,0
dbfs:/tmp/unmanagedTable/_committed_7196093557433898980,_committed_7196093557433898980,226
dbfs:/tmp/unmanagedTable/_started_7196093557433898980,_started_7196093557433898980,0
dbfs:/tmp/unmanagedTable/part-00000-tid-7196093557433898980-bfe69b68-43b4-444a-82a0-fc8e95dd6d4f-2225-1-c000.snappy.parquet,part-00000-tid-7196093557433898980-bfe69b68-43b4-444a-82a0-fc8e95dd6d4f-2225-1-c000.snappy.parquet,716
dbfs:/tmp/unmanagedTable/part-00001-tid-7196093557433898980-bfe69b68-43b4-444a-82a0-fc8e95dd6d4f-2226-1-c000.snappy.parquet,part-00001-tid-7196093557433898980-bfe69b68-43b4-444a-82a0-fc8e95dd6d4f-2226-1-c000.snappy.parquet,716


## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Dropping Managed and Unmanaged Tables<br>

Confirm that the underlying files exist for the managed table.

In [0]:
%fs ls dbfs:/user/hive/warehouse/tablemanaged

path,name,size
dbfs:/user/hive/warehouse/tablemanaged/_delta_log/,_delta_log/,0
dbfs:/user/hive/warehouse/tablemanaged/part-00000-55fd19e1-c676-4fa7-a543-361c5bc22df7-c000.snappy.parquet,part-00000-55fd19e1-c676-4fa7-a543-361c5bc22df7-c000.snappy.parquet,730
dbfs:/user/hive/warehouse/tablemanaged/part-00001-e6bf94d1-bbb5-4c25-8ccd-728d4734ecc6-c000.snappy.parquet,part-00001-e6bf94d1-bbb5-4c25-8ccd-728d4734ecc6-c000.snappy.parquet,730


Now drop the managed table.

In [0]:
%sql
DROP TABLE tableManaged

Take a look--the files are gone!  (Uncomment the code to run)

In [0]:
%sql
%fs ls dbfs:/user/hive/warehouse/tablemanaged

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.catalyst.parser.ParseException: 
mismatched input '%' expecting {'(', 'CONVERT', 'COPY', 'OPTIMIZE', 'RESTORE', 'ADD', 'ALTER', 'ANALYZE', 'CACHE', 'CLEAR', 'COMMENT', 'COMMIT', 'CREATE', 'DELETE', 'DESC', 'DESCRIBE', 'DFS', 'DROP', 'EXPLAIN', 'EXPORT', 'FROM', 'GRANT', 'IMPORT', 'INSERT', 'LIST', 'LOAD', 'LOCK', 'MAP', 'MERGE', 'MSCK', 'REDUCE', 'REFRESH', 'REPLACE', 'RESET', 'REVOKE', 'ROLLBACK', 'SELECT', 'SET', 'SHOW', 'START', 'TABLE', 'TRUNCATE', 'UNCACHE', 'UNLOCK', 'UPDATE', 'USE', 'VALUES', 'WITH'}(line 1, pos 0)

== SQL ==
%fs ls dbfs:/user/hive/warehouse/tablemanaged
^^^

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(ParseDriver.scala:265)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parse(ParseDriver.scala:134)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:64)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parsePlan(ParseDriver.scala:85)
	at com.databricks.sql.parser.DatabricksSqlParser.$anonfun$parsePlan$1(DatabricksSqlParser.scala:67)
	at com.databricks.sql.parser.DatabricksSqlParser.parse(DatabricksSqlParser.scala:87)
	at com.databricks.sql.parser.DatabricksSqlParser.parsePlan(DatabricksSqlParser.scala:64)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$2(SparkSession.scala:687)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:134)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:687)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:854)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:684)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:694)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:91)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.immutable.List.map(List.scala:298)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:37)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:145)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$11(DriverLocal.scala:526)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:266)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:261)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:258)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:50)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:305)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:297)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:50)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:503)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:689)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:681)
	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:522)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:634)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:427)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:370)
	at com.databricks.bac

Now drop the unmanaged, or external, table.

In [0]:
%sql
DROP TABLE tableUnmanaged

Now take a look at the underlying files.

In [0]:
%fs ls /tmp/unmanagedTable

path,name,size
dbfs:/tmp/unmanagedTable/_SUCCESS,_SUCCESS,0
dbfs:/tmp/unmanagedTable/_committed_7196093557433898980,_committed_7196093557433898980,226
dbfs:/tmp/unmanagedTable/_started_7196093557433898980,_started_7196093557433898980,0
dbfs:/tmp/unmanagedTable/part-00000-tid-7196093557433898980-bfe69b68-43b4-444a-82a0-fc8e95dd6d4f-2225-1-c000.snappy.parquet,part-00000-tid-7196093557433898980-bfe69b68-43b4-444a-82a0-fc8e95dd6d4f-2225-1-c000.snappy.parquet,716
dbfs:/tmp/unmanagedTable/part-00001-tid-7196093557433898980-bfe69b68-43b4-444a-82a0-fc8e95dd6d4f-2226-1-c000.snappy.parquet,part-00001-tid-7196093557433898980-bfe69b68-43b4-444a-82a0-fc8e95dd6d4f-2226-1-c000.snappy.parquet,716


They're still there!

## Summary
- Use the appropriate view or table based on how you want to persist your query
- Use external/unmanaged tables when you want to persist your data once the cluster has shut down
- Use managed tables when you only want ephemeral data

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>